# MLP Classification with TREC Dataset
<hr>

We will build a text classification model using MLP model on the TREC Dataset. 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from nltk.corpus import stopwords, twitter_samples
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import time

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

## Load the Dataset

In [2]:
corpus = pd.read_pickle('../../../0_data/TREC/TREC.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(5952, 3)


sentence  label  split
0     how did serfdom develop in and then leave russ...      0  train
1      what films featured the character popeye doyle ?      1  train
2     how can i find a list of celebrities ' real na...      0  train
3     what fowl grabs the spotlight after the chines...      1  train
4                       what is the full form of .com ?      2  train
...                                                 ...    ...    ...
5947             who was the 22nd president of the us ?      3   test
5948             what is the money they use in zambia ?      1   test
5949                          how many feet in a mile ?      5   test
5950                what is the birthstone of october ?      1   test
5951                                   what is e coli ?      0   test

[5952 rows x 3 columns]

In [3]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5952 non-null   object
 1   label     5952 non-null   int32 
 2   split     5952 non-null   object
dtypes: int32(1), object(2)
memory usage: 116.4+ KB


In [4]:
corpus.groupby( by=['split','label']).count()

sentence
split label          
test  0           138
      1            94
      2             9
      3            65
      4            81
      5           113
train 0          1162
      1          1250
      2            86
      3          1223
      4           835
      5           896

In [5]:
corpus.groupby(by='split').count()

sentence  label
split                 
test        500    500
train      5452   5452

In [6]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

5452
5452
500
500


# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>## Develop Vocabulary

A part of preparing text for text classification involves defining and tailoring the vocabulary of words supported by the model. **We can do this by loading all of the documents in the dataset and building a set of words.**

The larger the vocabulary, the more sparse the representation of each word or document. So, we may decide to support all of these words, or perhaps discard some. The final chosen vocabulary can then be saved to a file for later use, such as filtering words in new documents in the future.

In [7]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  what is the full form of .com ?
Into a sequence of int: [3, 4, 2, 471, 261, 5, 372]
Into a padded sequence: [  3   4   2 471 261   5 372   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [9]:
# See the first 10 words in the vocabulary

word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
what 3
is 4
of 5
in 6
a 7
how 8
's 9
was 10
8461


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 1-8.
- Number of filters: 100.
- Dropout rate: 0.5.
- L2 Constraint: 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [10]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [32]:
def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(units=6, activation='softmax')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length, mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(units=6, activation='softmax')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(units=6, activation='softmax')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [33]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_46 (Embedding)        (None, 100, 300)     300000      input_47[0][0]                   
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 98, 100)      90100       embedding_46[0][0]               
__________________________________________________________________________________________________
max_pooling1d_23 (MaxPooling1D) (None, 49, 100)      0           conv1d_23[0][0]                  
___________________________________________________________________________________________

In [34]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)
# callbacks = myCallback()

## Train and Test the Model

In [35]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'Acc']
record = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

for activation in activations:

    for kernel_size in kernel_sizes:

        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1

        # Define the model
        model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

        # Train the model
        acc = 0 
        while (acc<0.85):
            
            model.fit([Xtrain, Xtrain], train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))
            
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            
            if (acc<0.7):
                # Re-define the model
                model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)
                print('The model is suffered from optimizing the local minimum :(')
                
            else:
                print('Done!')

        # evaluate the model
        loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
        print('Test Accuracy: {}'.format(acc*100))

        parameters = [activation, kernel_size]
        entries = parameters + [acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

Epoch 1/100
110/110 [==============================] - 33s 155ms/step - loss: 1.6845 - accuracy: 0.2857 - val_loss: 1.3982 - val_accuracy: 0.5000
Epoch 2/100
110/110 [==============================] - 17s 157ms/step - loss: 1.2695 - accuracy: 0.6811 - val_loss: 1.1214 - val_accuracy: 0.8400
Epoch 3/100
110/110 [==============================] - 15s 135ms/step - loss: 1.0042 - accuracy: 0.8732 - val_loss: 0.9079 - val_accuracy: 0.8360
Epoch 4/100
110/110 [==============================] - 14s 131ms/step - loss: 0.7675 - accuracy: 0.9236 - val_loss: 0.8332 - val_accuracy: 0.8420
Epoch 5/100
110/110 [==============================] - 13s 119ms/step - loss: 0.6072 - accuracy: 0.9392 - val_loss: 0.7146 - val_accuracy: 0.8340
Epoch 6/100
110/110 [==============================] - 14s 124ms/step - loss: 0.4988 - accuracy: 0.9470 - val_loss: 0.6558 - val_accuracy: 0.8500
Epoch 7/100
110/110 [==============================] - 14s 128ms/step - loss: 0.4226 - accuracy: 0.9505 - val_loss: 0.6518 -

110/110 [==============================] - 15s 134ms/step - loss: 0.0982 - accuracy: 0.9919 - val_loss: 0.6070 - val_accuracy: 0.8220
Epoch 25/100
110/110 [==============================] - 14s 130ms/step - loss: 0.0925 - accuracy: 0.9902 - val_loss: 0.5649 - val_accuracy: 0.8660
Epoch 26/100
110/110 [==============================] - 15s 132ms/step - loss: 0.0853 - accuracy: 0.9928 - val_loss: 0.5499 - val_accuracy: 0.8460
Epoch 27/100
110/110 [==============================] - 15s 136ms/step - loss: 0.0779 - accuracy: 0.9932 - val_loss: 0.5392 - val_accuracy: 0.8620
Epoch 28/100
110/110 [==============================] - 14s 132ms/step - loss: 0.0817 - accuracy: 0.9931 - val_loss: 0.5585 - val_accuracy: 0.8620
Epoch 29/100
110/110 [==============================] - 14s 126ms/step - loss: 0.0716 - accuracy: 0.9932 - val_loss: 0.5986 - val_accuracy: 0.8360
Epoch 30/100
110/110 [==============================] - 14s 132ms/step - loss: 0.0641 - accuracy: 0.9944 - val_loss: 0.6136 - val_a

110/110 [==============================] - 14s 126ms/step - loss: 0.1749 - accuracy: 0.9629 - val_loss: 0.5882 - val_accuracy: 0.8440
Epoch 25/100
110/110 [==============================] - 14s 126ms/step - loss: 0.1587 - accuracy: 0.9635 - val_loss: 0.6233 - val_accuracy: 0.8340
Epoch 26/100
110/110 [==============================] - 15s 133ms/step - loss: 0.1739 - accuracy: 0.9582 - val_loss: 0.5920 - val_accuracy: 0.8480
Epoch 27/100
110/110 [==============================] - 14s 129ms/step - loss: 0.1521 - accuracy: 0.9639 - val_loss: 0.6192 - val_accuracy: 0.8380
Epoch 28/100
110/110 [==============================] - 13s 121ms/step - loss: 0.1623 - accuracy: 0.9559 - val_loss: 0.6763 - val_accuracy: 0.8180
Restoring model weights from the end of the best epoch.
Epoch 00028: early stopping
Done!
Test Accuracy: 86.40000224113464

  Activation Filters    Acc
0       relu       1  0.864
1       relu       2  0.880
2       relu       3  0.864

Epoch 1/100
110/110 [====================

110/110 [==============================] - 31s 156ms/step - loss: 1.7965 - accuracy: 0.1757 - val_loss: 1.3777 - val_accuracy: 0.6100
Epoch 2/100
110/110 [==============================] - 14s 129ms/step - loss: 1.3515 - accuracy: 0.6353 - val_loss: 1.1043 - val_accuracy: 0.7140
Epoch 3/100
110/110 [==============================] - 17s 155ms/step - loss: 1.0481 - accuracy: 0.7746 - val_loss: 0.9692 - val_accuracy: 0.8380
Epoch 4/100
110/110 [==============================] - 15s 138ms/step - loss: 0.8551 - accuracy: 0.9018 - val_loss: 0.8761 - val_accuracy: 0.8480
Epoch 5/100
110/110 [==============================] - 15s 136ms/step - loss: 0.7235 - accuracy: 0.9193 - val_loss: 0.7800 - val_accuracy: 0.8620
Epoch 6/100
110/110 [==============================] - 15s 137ms/step - loss: 0.6412 - accuracy: 0.9096 - val_loss: 0.7065 - val_accuracy: 0.8600
Epoch 7/100
110/110 [==============================] - 15s 140ms/step - loss: 0.5416 - accuracy: 0.9360 - val_loss: 0.7011 - val_accurac

Epoch 15/100
110/110 [==============================] - 13s 121ms/step - loss: 0.1679 - accuracy: 0.9748 - val_loss: 0.4927 - val_accuracy: 0.8560
Epoch 16/100
110/110 [==============================] - 13s 122ms/step - loss: 0.1551 - accuracy: 0.9745 - val_loss: 0.4953 - val_accuracy: 0.8600
Epoch 17/100
110/110 [==============================] - 13s 121ms/step - loss: 0.1479 - accuracy: 0.9731 - val_loss: 0.5105 - val_accuracy: 0.8500
Epoch 18/100
110/110 [==============================] - 14s 124ms/step - loss: 0.1239 - accuracy: 0.9846 - val_loss: 0.5131 - val_accuracy: 0.8620
Epoch 19/100
110/110 [==============================] - 13s 121ms/step - loss: 0.1292 - accuracy: 0.9847 - val_loss: 0.4740 - val_accuracy: 0.8720
Epoch 20/100
110/110 [==============================] - 14s 126ms/step - loss: 0.1154 - accuracy: 0.9866 - val_loss: 0.5115 - val_accuracy: 0.8660
Epoch 21/100
110/110 [==============================] - 14s 123ms/step - loss: 0.1050 - accuracy: 0.9887 - val_loss: 0

## Summary

In [36]:
record.sort_values(by='Acc', ascending=False)

Activation Filters    Acc
1       relu       2  0.880
3       relu       4  0.876
4       relu       5  0.874
5       relu       6  0.872
0       relu       1  0.864
2       relu       3  0.864

In [37]:
report = record.sort_values(by='Acc', ascending=False)
report = report.to_excel('HYBRID_TREC.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [38]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [39]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [40]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [41]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 7526 words present from 8761 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [42]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [43]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [44]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## CNN Model

In [45]:
def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(units=6, activation='softmax')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(units=6, activation='softmax')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(units=6, activation='softmax')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [46]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_61 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_60 (Embedding)        (None, 100, 300)     300000      input_61[0][0]                   
__________________________________________________________________________________________________
conv1d_30 (Conv1D)              (None, 98, 100)      90100       embedding_60[0][0]               
__________________________________________________________________________________________________
max_pooling1d_30 (MaxPooling1D) (None, 49, 100)      0           conv1d_30[0][0]                  
___________________________________________________________________________________________

## Train and Test the Model

In [47]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=30, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [48]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record2 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

for activation in activations:
    for kernel_size in kernel_sizes:
            
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1
        
        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
        
        # Define the input shape
        model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                              max_length=max_len, emb_matrix=emb_matrix)

        # Train the model
        acc = 0 
        while (acc<0.85):
            
            model.fit([Xtrain, Xtrain], train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))
            
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            
            if (acc<0.7):
                # Re-define the model
                model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
                print('The model is suffered from optimizing the local minimum :(')
                
            else:
                print('Done!')

        # evaluate the model
        loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
        print('Test Accuracy: {}'.format(acc*100))
            
        parameters = [activation, kernel_size]
        entries = parameters + [acc*100]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Epoch 1/100
110/110 [==============================] - 25s 98ms/step - loss: 1.7059 - accuracy: 0.2474 - val_loss: 1.5266 - val_accuracy: 0.4800
Epoch 2/100
110/110 [==============================] - 7s 63ms/step - loss: 1.4838 - accuracy: 0.4814 - val_loss: 1.3780 - val_accuracy: 0.5540
Epoch 3/100
110/110 [==============================] - 7s 65ms/step - loss: 1.3362 - accuracy: 0.5428 - val_loss: 1.2007 - val_accuracy: 0.7380
Epoch 4/100
110/110 [==============================] - 7s 63ms/step - loss: 1.2001 - accuracy: 0.6386 - val_loss: 1.0186 - val_accuracy: 0.8120
Epoch 5/100
110/110 [==============================] - 7s 63ms/step - loss: 1.0487 - accuracy: 0.7232 - val_loss: 0.8937 - val_accuracy: 0.8120
Epoch 6/100
110/110 [==============================] - 7s 63ms/step - loss: 0.9167 - accuracy: 0.7750 - val_loss: 0.7844 - val_accuracy: 0.8420
Epoch 7/100
110/110 [==============================] - 7s 62ms/step - loss: 0.8305 - accuracy: 0.7980 - val_loss: 0.6991 - val_accuracy

110/110 [==============================] - 7s 62ms/step - loss: 0.1617 - accuracy: 0.9477 - val_loss: 0.3507 - val_accuracy: 0.9060
Epoch 58/100
110/110 [==============================] - 7s 63ms/step - loss: 0.1503 - accuracy: 0.9499 - val_loss: 0.3460 - val_accuracy: 0.9020
Epoch 59/100
110/110 [==============================] - 7s 64ms/step - loss: 0.1591 - accuracy: 0.9478 - val_loss: 0.3734 - val_accuracy: 0.9000
Epoch 60/100
110/110 [==============================] - 7s 63ms/step - loss: 0.1445 - accuracy: 0.9507 - val_loss: 0.3574 - val_accuracy: 0.9080
Epoch 61/100
110/110 [==============================] - 8s 70ms/step - loss: 0.1559 - accuracy: 0.9490 - val_loss: 0.3532 - val_accuracy: 0.9120
Epoch 62/100
110/110 [==============================] - 9s 82ms/step - loss: 0.1383 - accuracy: 0.9571 - val_loss: 0.3703 - val_accuracy: 0.8980
Epoch 63/100
110/110 [==============================] - 9s 81ms/step - loss: 0.1456 - accuracy: 0.9521 - val_loss: 0.3738 - val_accuracy: 0.906

110/110 [==============================] - 7s 65ms/step - loss: 0.1615 - accuracy: 0.9502 - val_loss: 0.3528 - val_accuracy: 0.9080
Epoch 31/100
110/110 [==============================] - 7s 64ms/step - loss: 0.1588 - accuracy: 0.9531 - val_loss: 0.3233 - val_accuracy: 0.9180
Epoch 32/100
110/110 [==============================] - 7s 64ms/step - loss: 0.1597 - accuracy: 0.9609 - val_loss: 0.3432 - val_accuracy: 0.9100
Epoch 33/100
110/110 [==============================] - 7s 67ms/step - loss: 0.1511 - accuracy: 0.9548 - val_loss: 0.3401 - val_accuracy: 0.9120
Epoch 34/100
110/110 [==============================] - 7s 64ms/step - loss: 0.1482 - accuracy: 0.9607 - val_loss: 0.3749 - val_accuracy: 0.9020
Epoch 35/100
110/110 [==============================] - 7s 66ms/step - loss: 0.1583 - accuracy: 0.9631 - val_loss: 0.3648 - val_accuracy: 0.9120
Epoch 36/100
110/110 [==============================] - 9s 80ms/step - loss: 0.1404 - accuracy: 0.9606 - val_loss: 0.3913 - val_accuracy: 0.894

110/110 [==============================] - 7s 66ms/step - loss: 0.2066 - accuracy: 0.9523 - val_loss: 0.4273 - val_accuracy: 0.8900
Epoch 25/100
110/110 [==============================] - 7s 63ms/step - loss: 0.2068 - accuracy: 0.9489 - val_loss: 0.4039 - val_accuracy: 0.8860
Epoch 26/100
110/110 [==============================] - 8s 75ms/step - loss: 0.2046 - accuracy: 0.9496 - val_loss: 0.3717 - val_accuracy: 0.8940
Epoch 27/100
110/110 [==============================] - 9s 80ms/step - loss: 0.1883 - accuracy: 0.9509 - val_loss: 0.4048 - val_accuracy: 0.8880
Epoch 28/100
110/110 [==============================] - 7s 64ms/step - loss: 0.1878 - accuracy: 0.9517 - val_loss: 0.3959 - val_accuracy: 0.8860
Epoch 29/100
110/110 [==============================] - 7s 67ms/step - loss: 0.1848 - accuracy: 0.9526 - val_loss: 0.4301 - val_accuracy: 0.8840
Epoch 30/100
110/110 [==============================] - 7s 66ms/step - loss: 0.1834 - accuracy: 0.9522 - val_loss: 0.3767 - val_accuracy: 0.892

110/110 [==============================] - 8s 71ms/step - loss: 0.0903 - accuracy: 0.9756 - val_loss: 0.4016 - val_accuracy: 0.9020
Epoch 81/100
110/110 [==============================] - 8s 69ms/step - loss: 0.0919 - accuracy: 0.9755 - val_loss: 0.4290 - val_accuracy: 0.8960
Epoch 82/100
110/110 [==============================] - 7s 66ms/step - loss: 0.0836 - accuracy: 0.9760 - val_loss: 0.4773 - val_accuracy: 0.9020
Epoch 83/100
110/110 [==============================] - 7s 64ms/step - loss: 0.0912 - accuracy: 0.9753 - val_loss: 0.4746 - val_accuracy: 0.8980
Restoring model weights from the end of the best epoch.
Epoch 00083: early stopping
Done!
Test Accuracy: 92.79999732971191

  Activation Filters        Acc
0       relu       1  91.399997
1       relu       2  91.799998
2       relu       3  92.799997

Epoch 1/100
110/110 [==============================] - 25s 96ms/step - loss: 1.6981 - accuracy: 0.2671 - val_loss: 1.4761 - val_accuracy: 0.6040
Epoch 2/100
110/110 [==============

Epoch 52/100
110/110 [==============================] - 7s 65ms/step - loss: 0.0778 - accuracy: 0.9807 - val_loss: 0.3107 - val_accuracy: 0.9260
Epoch 53/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0717 - accuracy: 0.9842 - val_loss: 0.3756 - val_accuracy: 0.9180
Epoch 54/100
110/110 [==============================] - 7s 61ms/step - loss: 0.0790 - accuracy: 0.9786 - val_loss: 0.4059 - val_accuracy: 0.8840
Epoch 55/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0715 - accuracy: 0.9838 - val_loss: 0.4571 - val_accuracy: 0.8320
Epoch 56/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0743 - accuracy: 0.9817 - val_loss: 0.3965 - val_accuracy: 0.8880
Epoch 57/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0663 - accuracy: 0.9842 - val_loss: 0.4182 - val_accuracy: 0.8960
Epoch 58/100
110/110 [==============================] - 7s 61ms/step - loss: 0.0603 - accuracy: 0.9859 - val_loss: 0.2956 - val_ac

110/110 [==============================] - 7s 62ms/step - loss: 0.0796 - accuracy: 0.9817 - val_loss: 0.3769 - val_accuracy: 0.9000
Epoch 47/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0682 - accuracy: 0.9825 - val_loss: 0.3888 - val_accuracy: 0.8960
Epoch 48/100
110/110 [==============================] - 7s 64ms/step - loss: 0.0771 - accuracy: 0.9780 - val_loss: 0.3949 - val_accuracy: 0.9020
Epoch 49/100
110/110 [==============================] - 7s 63ms/step - loss: 0.0695 - accuracy: 0.9816 - val_loss: 0.4051 - val_accuracy: 0.9020
Epoch 50/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0721 - accuracy: 0.9817 - val_loss: 0.4368 - val_accuracy: 0.8920
Epoch 51/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0632 - accuracy: 0.9846 - val_loss: 0.4209 - val_accuracy: 0.8980
Epoch 52/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0653 - accuracy: 0.9792 - val_loss: 0.4022 - val_accuracy: 0.894

110/110 [==============================] - 7s 62ms/step - loss: 0.1228 - accuracy: 0.9741 - val_loss: 0.4451 - val_accuracy: 0.8880
Epoch 36/100
110/110 [==============================] - 7s 62ms/step - loss: 0.0992 - accuracy: 0.9846 - val_loss: 0.4293 - val_accuracy: 0.8880
Epoch 37/100
110/110 [==============================] - 7s 65ms/step - loss: 0.0841 - accuracy: 0.9881 - val_loss: 0.3981 - val_accuracy: 0.9000
Epoch 38/100
110/110 [==============================] - 7s 65ms/step - loss: 0.1003 - accuracy: 0.9827 - val_loss: 0.4736 - val_accuracy: 0.8720
Epoch 39/100
110/110 [==============================] - 7s 65ms/step - loss: 0.0875 - accuracy: 0.9854 - val_loss: 0.3895 - val_accuracy: 0.8960
Epoch 40/100
110/110 [==============================] - 7s 67ms/step - loss: 0.0813 - accuracy: 0.9848 - val_loss: 0.3887 - val_accuracy: 0.9060
Epoch 41/100
110/110 [==============================] - 7s 66ms/step - loss: 0.0866 - accuracy: 0.9843 - val_loss: 0.4327 - val_accuracy: 0.898

## Summary

In [49]:
record2.sort_values(by='Acc', ascending=False)

Activation Filters        Acc
3       relu       4  93.000001
2       relu       3  92.799997
1       relu       2  91.799998
5       relu       6  91.799998
0       relu       1  91.399997
4       relu       5  91.200000

In [50]:
report = record2.sort_values(by='Acc', ascending=False)
report = report.to_excel('HYBRID_TREC_2.xlsx', sheet_name='static')

# Model 3: Word2Vec Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [51]:
def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(units=6, activation='softmax')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(units=6, activation='softmax')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(units=6, activation='softmax')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [52]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_37"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_75 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_74 (Embedding)        (None, 100, 300)     300000      input_75[0][0]                   
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 98, 100)      90100       embedding_74[0][0]               
__________________________________________________________________________________________________
max_pooling1d_37 (MaxPooling1D) (None, 49, 100)      0           conv1d_37[0][0]                  
___________________________________________________________________________________________

## Train and Test the Model

In [53]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=30, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [54]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'Acc']
record3 = pd.DataFrame(columns = columns)

# Separate the sentences and the labels
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)

for activation in activations:
    for kernel_size in kernel_sizes:
            
        # encode data using
        # Cleaning and Tokenization
        tokenizer = Tokenizer(oov_token=oov_tok)
        tokenizer.fit_on_texts(train_x)

        # Turn the text into sequence
        training_sequences = tokenizer.texts_to_sequences(train_x)
        test_sequences = tokenizer.texts_to_sequences(test_x)

        max_len = max_length(training_sequences)

        # Pad the sequence to have the same size
        Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
        Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

        word_index = tokenizer.word_index
        vocab_size = len(word_index)+1
        
        emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
        
        # Define the input shape
        model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                              max_length=max_len, emb_matrix=emb_matrix)

        # Train the model
        acc = 0 
        while (acc<0.85):
            
            model.fit([Xtrain, Xtrain], train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))
            
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            
            if (acc<0.7):
                # Re-define the model
                model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                       max_length=max_len, emb_matrix=emb_matrix)
                print('The model is suffered from optimizing the local minimum :(')
                
            else:
                print('Done!')

        # evaluate the model
        loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
        print('Test Accuracy: {}'.format(acc*100))
            
        parameters = [activation, kernel_size]
        entries = parameters + [acc*100]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Epoch 1/100
110/110 [==============================] - 81s 147ms/step - loss: 1.6744 - accuracy: 0.3014 - val_loss: 1.4577 - val_accuracy: 0.3480
Epoch 2/100
110/110 [==============================] - 13s 118ms/step - loss: 1.3593 - accuracy: 0.5510 - val_loss: 1.2436 - val_accuracy: 0.6360
Epoch 3/100
110/110 [==============================] - 13s 116ms/step - loss: 1.1638 - accuracy: 0.6565 - val_loss: 1.0714 - val_accuracy: 0.6740
Epoch 4/100
110/110 [==============================] - 13s 116ms/step - loss: 0.9866 - accuracy: 0.7447 - val_loss: 0.9237 - val_accuracy: 0.7360
Epoch 5/100
110/110 [==============================] - 13s 116ms/step - loss: 0.8598 - accuracy: 0.7699 - val_loss: 0.8451 - val_accuracy: 0.7520
Epoch 6/100
110/110 [==============================] - 13s 116ms/step - loss: 0.7280 - accuracy: 0.8416 - val_loss: 0.6887 - val_accuracy: 0.8760
Epoch 7/100
110/110 [==============================] - 13s 115ms/step - loss: 0.5655 - accuracy: 0.9414 - val_loss: 0.6144 -

110/110 [==============================] - 13s 122ms/step - loss: 0.1733 - accuracy: 0.9855 - val_loss: 0.4871 - val_accuracy: 0.8840
Epoch 18/100
110/110 [==============================] - 13s 117ms/step - loss: 0.1580 - accuracy: 0.9845 - val_loss: 0.4857 - val_accuracy: 0.8820
Epoch 19/100
110/110 [==============================] - 13s 122ms/step - loss: 0.1461 - accuracy: 0.9882 - val_loss: 0.5048 - val_accuracy: 0.8700
Epoch 20/100
110/110 [==============================] - 13s 121ms/step - loss: 0.1400 - accuracy: 0.9880 - val_loss: 0.5471 - val_accuracy: 0.8520
Epoch 21/100
110/110 [==============================] - 13s 118ms/step - loss: 0.1330 - accuracy: 0.9858 - val_loss: 0.4792 - val_accuracy: 0.8860
Epoch 22/100
110/110 [==============================] - 13s 116ms/step - loss: 0.1250 - accuracy: 0.9869 - val_loss: 0.4628 - val_accuracy: 0.8820
Epoch 23/100
110/110 [==============================] - 13s 119ms/step - loss: 0.1257 - accuracy: 0.9842 - val_loss: 0.5008 - val_a

110/110 [==============================] - 13s 116ms/step - loss: 0.1582 - accuracy: 0.9712 - val_loss: 0.6203 - val_accuracy: 0.8540
Epoch 29/100
110/110 [==============================] - 13s 114ms/step - loss: 0.1560 - accuracy: 0.9763 - val_loss: 0.6192 - val_accuracy: 0.8580
Epoch 30/100
110/110 [==============================] - 13s 114ms/step - loss: 0.1331 - accuracy: 0.9838 - val_loss: 0.6233 - val_accuracy: 0.8600
Epoch 31/100
110/110 [==============================] - 13s 119ms/step - loss: 0.1248 - accuracy: 0.9831 - val_loss: 0.6124 - val_accuracy: 0.8600
Epoch 32/100
110/110 [==============================] - 13s 118ms/step - loss: 0.1324 - accuracy: 0.9806 - val_loss: 0.6302 - val_accuracy: 0.8560
Epoch 33/100
110/110 [==============================] - 13s 120ms/step - loss: 0.1224 - accuracy: 0.9832 - val_loss: 0.6266 - val_accuracy: 0.8500
Epoch 34/100
110/110 [==============================] - 13s 118ms/step - loss: 0.1208 - accuracy: 0.9818 - val_loss: 0.6405 - val_a

110/110 [==============================] - 13s 119ms/step - loss: 0.4259 - accuracy: 0.9473 - val_loss: 0.6046 - val_accuracy: 0.8720
Epoch 10/100
110/110 [==============================] - 13s 117ms/step - loss: 0.3813 - accuracy: 0.9483 - val_loss: 0.6293 - val_accuracy: 0.8480
Epoch 11/100
110/110 [==============================] - 13s 117ms/step - loss: 0.3468 - accuracy: 0.9511 - val_loss: 0.5748 - val_accuracy: 0.8680
Epoch 12/100
110/110 [==============================] - 13s 117ms/step - loss: 0.3111 - accuracy: 0.9569 - val_loss: 0.5639 - val_accuracy: 0.8640
Epoch 13/100
110/110 [==============================] - 13s 116ms/step - loss: 0.2887 - accuracy: 0.9579 - val_loss: 0.5463 - val_accuracy: 0.8700
Epoch 14/100
110/110 [==============================] - 13s 117ms/step - loss: 0.2541 - accuracy: 0.9710 - val_loss: 0.5377 - val_accuracy: 0.8780
Epoch 15/100
110/110 [==============================] - 13s 118ms/step - loss: 0.2407 - accuracy: 0.9752 - val_loss: 0.5633 - val_a

Epoch 19/100
110/110 [==============================] - 13s 116ms/step - loss: 0.1673 - accuracy: 0.9722 - val_loss: 0.6672 - val_accuracy: 0.8160
Epoch 20/100
110/110 [==============================] - 13s 115ms/step - loss: 0.1598 - accuracy: 0.9704 - val_loss: 0.5024 - val_accuracy: 0.8640
Epoch 21/100
110/110 [==============================] - 13s 116ms/step - loss: 0.1456 - accuracy: 0.9746 - val_loss: 0.5629 - val_accuracy: 0.8500
Epoch 22/100
110/110 [==============================] - 13s 116ms/step - loss: 0.1395 - accuracy: 0.9728 - val_loss: 0.5605 - val_accuracy: 0.8400
Epoch 23/100
110/110 [==============================] - 13s 115ms/step - loss: 0.1232 - accuracy: 0.9772 - val_loss: 0.5352 - val_accuracy: 0.8520
Epoch 24/100
110/110 [==============================] - 13s 116ms/step - loss: 0.1295 - accuracy: 0.9732 - val_loss: 0.5808 - val_accuracy: 0.8460
Epoch 25/100
110/110 [==============================] - 13s 116ms/step - loss: 0.1285 - accuracy: 0.9818 - val_loss: 0

110/110 [==============================] - 14s 128ms/step - loss: 0.1610 - accuracy: 0.9751 - val_loss: 0.6230 - val_accuracy: 0.8460
Epoch 25/100
110/110 [==============================] - 14s 127ms/step - loss: 0.1427 - accuracy: 0.9780 - val_loss: 0.5802 - val_accuracy: 0.8660
Epoch 26/100
110/110 [==============================] - 14s 129ms/step - loss: 0.1412 - accuracy: 0.9835 - val_loss: 0.6101 - val_accuracy: 0.8600
Epoch 27/100
110/110 [==============================] - 15s 138ms/step - loss: 0.1333 - accuracy: 0.9854 - val_loss: 0.6478 - val_accuracy: 0.8500
Epoch 28/100
110/110 [==============================] - 14s 124ms/step - loss: 0.1297 - accuracy: 0.9847 - val_loss: 0.6248 - val_accuracy: 0.8600
Epoch 29/100
110/110 [==============================] - 15s 139ms/step - loss: 0.1318 - accuracy: 0.9835 - val_loss: 0.6858 - val_accuracy: 0.8380
Epoch 30/100
110/110 [==============================] - 15s 136ms/step - loss: 0.1160 - accuracy: 0.9897 - val_loss: 0.6829 - val_a

110/110 [==============================] - 13s 118ms/step - loss: 0.0664 - accuracy: 0.9933 - val_loss: 0.5935 - val_accuracy: 0.8620
Epoch 35/100
110/110 [==============================] - 13s 118ms/step - loss: 0.0694 - accuracy: 0.9923 - val_loss: 0.6172 - val_accuracy: 0.8660
Epoch 36/100
110/110 [==============================] - 13s 120ms/step - loss: 0.0588 - accuracy: 0.9938 - val_loss: 0.6332 - val_accuracy: 0.8640
Epoch 37/100
110/110 [==============================] - 14s 129ms/step - loss: 0.0618 - accuracy: 0.9936 - val_loss: 0.6043 - val_accuracy: 0.8700
Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
Done!
Test Accuracy: 88.59999775886536

  Activation Filters        Acc
0       relu       1  88.800001
1       relu       2  88.800001
2       relu       3  87.000000
3       relu       4  87.800002
4       relu       5  86.400002
5       relu       6  87.400001
6       relu       7  88.599998

Epoch 1/100
110/110 [=======================

110/110 [==============================] - 13s 123ms/step - loss: 0.0331 - accuracy: 0.9956 - val_loss: 0.5835 - val_accuracy: 0.8640
Epoch 51/100
110/110 [==============================] - 14s 124ms/step - loss: 0.0284 - accuracy: 0.9969 - val_loss: 0.5778 - val_accuracy: 0.8760
Epoch 52/100
110/110 [==============================] - 14s 125ms/step - loss: 0.0277 - accuracy: 0.9976 - val_loss: 0.5864 - val_accuracy: 0.8720
Epoch 53/100
110/110 [==============================] - 14s 123ms/step - loss: 0.0311 - accuracy: 0.9966 - val_loss: 0.5874 - val_accuracy: 0.8760
Epoch 54/100
110/110 [==============================] - 13s 121ms/step - loss: 0.0335 - accuracy: 0.9948 - val_loss: 0.5992 - val_accuracy: 0.8780
Epoch 55/100
110/110 [==============================] - 13s 122ms/step - loss: 0.0271 - accuracy: 0.9961 - val_loss: 0.6018 - val_accuracy: 0.8780
Epoch 56/100
110/110 [==============================] - 13s 122ms/step - loss: 0.0283 - accuracy: 0.9965 - val_loss: 0.5945 - val_a

## Summary

In [55]:
record3.sort_values(by='Acc', ascending=False)

Activation Filters        Acc
7       relu       8  89.600003
0       relu       1  88.800001
1       relu       2  88.800001
6       relu       7  88.599998
3       relu       4  87.800002
5       relu       6  87.400001
2       relu       3  87.000000
4       relu       5  86.400002

In [56]:
report = record3.sort_values(by='Acc', ascending=False)
report = report.to_excel('HYBRID_TREC_3.xlsx', sheet_name='dynamic')